<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
<br>汉化的库: <a href="https://github.com/GoatCsu/CN-LLMs-from-scratch.git">https://github.com/GoatCsu/CN-LLMs-from-scratch.git</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


# **高效加载模型权重（Memory-efficient Model Weight Loading）**  

- 本笔记本提供了一些 **在 GPU（或 CPU）内存受限时加载大型预训练或微调模型的技巧**。  
- 具体来说，它重点介绍了 **如何加载使用 `torch.save(model.state_dict(), "model.pth")` 保存的模型**（例如在 **第 5-7 章** 中），以便在新会话中继续 **预训练（Pretraining）或额外微调（Finetuning）**。  
- **尽管示例使用的是 LLM**，但本笔记本介绍的方法 **适用于任何 PyTorch 模型的加载**，不仅限于 LLM。  

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/memory-efficient-loading/memory-efficient-loading.webp" width="800px">

In [1]:
from importlib.metadata import version

pkgs = [
    "torch",
]
for p in pkgs:
    print(f"{p} version: {version(p)}")

memory_profiler version: 0.61.0
torch version: 2.4.1+cu121


&nbsp;
## 1. **基准测试工具（Benchmark Utilities）**  

- 首先，我们定义一些 **用于追踪 VRAM（GPU 内存）** 的工具函数。  
- 之后，我们还将 **引入一个工具来监测主系统 RAM（CPU 内存）**。  
- 这些函数的作用将在后续应用时变得更加清晰。  

In [2]:
import gc
import time
import torch


def start_memory_tracking():
    """Initialize GPU memory tracking."""
    if torch.cuda.is_available():
        torch.cuda.reset_peak_memory_stats()
    else:
        print("This notebook is intended for CUDA GPUs but CUDA is not available.")

def print_memory_usage():
    max_gpu_memory = torch.cuda.max_memory_allocated() / (1024 ** 3)  # Convert bytes to GB
    print(f"Maximum GPU memory allocated: {max_gpu_memory:.1f} GB")

def cleanup():
    gc.collect()
    torch.cuda.empty_cache()
    time.sleep(3)  # some buffer time to allow memory to clear
    torch.cuda.reset_peak_memory_stats()
    max_memory_allocated = torch.cuda.max_memory_allocated(device) / (1024 ** 3)
    print(f"Maximum GPU memory allocated: {max_memory_allocated:.1f} GB")

&nbsp;
## 2. 模型设置

- 该代码部分 **用于初始化模型**。  
- 这里，我们使用 **"GPT-2 Large" 模型** 以增加实验的挑战性（如果希望减少 **内存占用** 和 **执行时间**，可以选择 **"GPT-2 Small (124M)"**）。  

In [3]:
from previous_chapters import GPTModel


BASE_CONFIG = {
    "vocab_size": 50257,     # Vocabulary size
    "context_length": 1024,  # Context length
    "drop_rate": 0.0,        # Dropout rate
    "qkv_bias": True         # Query-key-value bias
}

model_configs = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}

CHOOSE_MODEL = "gpt2-xl (1558M)"

BASE_CONFIG.update(model_configs[CHOOSE_MODEL])

- 现在，我们 **测试 GPU 内存监测函数的运行情况**。

In [4]:
start_memory_tracking()


model = GPTModel(BASE_CONFIG)
device = torch.device("cuda")
model.to(device)

print_memory_usage()

Maximum GPU memory allocated: 6.4 GB


- 此外，我们通过 **输入示例张量（tensor）** 来确保 **模型能够正常运行**。

In [5]:
# Test if the model works (no need to track memory here)
test_input = torch.tensor([[1, 2, 3]]).to(device)
model.eval()

with torch.no_grad():
    model(test_input)

- 接下来，假设我们 **完成了模型的预训练，并希望将其保存以便后续使用**。  
- **（为简洁起见，这里跳过实际的预训练过程，仅保存初始化的模型，但概念相同。）** 

In [6]:
# Training code would go here...

model.train()
torch.save(model.state_dict(), "model.pth")

- 最后，我们 **在 Python 会话中删除模型和示例张量，以释放 GPU 内存**。  

In [7]:
del model, test_input
cleanup()

Maximum GPU memory allocated: 0.0 GB


&nbsp;
## 3. 加载权重

- **接下来进入关键部分**，我们将 **加载预训练模型的权重**。  
- 让我们 **检查加载之前保存的模型所需的 GPU 内存**。  

In [8]:
# Then load pretrained weights

start_memory_tracking()

model = GPTModel(BASE_CONFIG)
model.to(device)

model.load_state_dict(
    torch.load("model.pth", map_location=device, weights_only=True)
)
model.to(device)
model.eval();

print_memory_usage()

Maximum GPU memory allocated: 12.8 GB


- **注意**，当前 **内存占用是前一阶段的 2 倍**。  
- 这是因为，在加载模型权重时，**短暂地在内存中存储了两份模型**：
  - **第一次**：通过 `model.to(device)` 将模型移动到设备（GPU/CPU）。  
  - **第二次**：调用  
    ```python
    model.load_state_dict(torch.load("model.pth", map_location=device, weights_only=True))
    ```  
    **在这一步，模型权重会被加载到 `state_dict` 中，然后复制到模型本体**。但在 **复制完成前**，**内存中同时存在完整的模型和加载的 `state_dict`**，导致占用翻倍。  
- **接下来的章节将介绍如何优化这一过程，以减少内存占用**。  
- 在此之前，我们先 **测试模型是否正确加载，并重置 GPU 内存**。  

In [9]:
# Test if the model works (no need to track memory here)
test_input = torch.tensor([[1, 2, 3]]).to(device)
model.eval()

with torch.no_grad():
    model(test_input)

del model, test_input
cleanup()

Maximum GPU memory allocated: 0.0 GB


&nbsp;
## **4. 按顺序加载权重（Loading Weights Sequentially）**  

- **为了解决上一节提到的“模型权重在 GPU 内存中出现两次”的问题**，可以采用 **按顺序加载（sequential loading）** 的方法。  
- 具体来说，我们 **分步加载模型**：
  1. **首先**，将 **模型架构** 加载到 **GPU 内存**。  
  2. **然后**，将 **模型权重** 先加载到 **CPU 内存**。  
  3. **最后**，逐个 **参数** 复制到 **GPU 内存**，避免一次性加载导致的内存峰值。  

---

In [10]:
start_memory_tracking()

model = GPTModel(BASE_CONFIG).to(device)

state_dict = torch.load("model.pth", map_location="cpu", weights_only=True)

print_memory_usage()

# Sequentially copy weights to the model's parameters
with torch.no_grad():
    for name, param in model.named_parameters():
        if name in state_dict:
            param.copy_(state_dict[name].to(device))
        else:
            print(f"Warning: {name} not found in state_dict.")

print_memory_usage()

Maximum GPU memory allocated: 6.4 GB
Maximum GPU memory allocated: 6.7 GB


- 如上所示，**采用按序加载方法后，内存占用明显降低**。  
- 需要注意的是，**内存从 6.4GB 增加到 6.7GB**，原因如下：  
  - **最初**，仅模型结构被加载到 **GPU 内存**。  
  - **随后**，每次加载一个参数张量（parameter tensor）并移动到 GPU，以便使用 `".to()"` 方法将其赋值到模型中。  
  - **整个过程中，我们只在 GPU 中存储一个额外的参数张量**，避免了大规模的额外占用。  
- **总体来看，这种方法显著降低了 GPU 内存峰值**。  
- 接下来，我们 **简单测试模型的正确性**，然后 **重置 GPU 内存，为下一节做准备**。  

In [11]:
# Test if the model works (no need to track memory here)
test_input = torch.tensor([[1, 2, 3]]).to(device)
model.eval()

with torch.no_grad():
    model(test_input)

del model, test_input, state_dict, param
cleanup()

Maximum GPU memory allocated: 0.0 GB


&nbsp;
## **5. 在低 CPU 内存环境中加载模型（Loading the Model with Low CPU Memory）**  

- 在上一节中，我们通过 **先将权重 (`state_dict`) 加载到 CPU 内存，再逐个复制到 GPU**，成功降低了 **GPU 内存占用**。  
- 但是，如果 **CPU 内存也有限**，我们该如何加载模型？  
- 本节将介绍 **PyTorch 的 `"meta"` 设备（Meta Device）方法**，该方法适用于 **GPU 内存充足但 CPU 内存较小的设备**。  
- 在此之前，我们先 **定义一个工具函数，用于监测 CPU 内存使用情况**。  

In [12]:
import os
import psutil
from threading import Thread


def memory_usage_in_gb(func, *args, **kwargs):
    process = psutil.Process(os.getpid())

    # Measure the baseline memory usage before running the function
    baseline_mem = process.memory_info().rss / 1024 ** 3  # in GB

    # Start monitoring memory in a separate thread
    mem_usage = []
    done = False

    def monitor_memory():
        while not done:
            mem_usage.append(process.memory_info().rss / 1024 ** 3)  # Convert to GB
            time.sleep(0.1)

    t = Thread(target=monitor_memory)
    t.start()

    # Run the function
    func(*args, **kwargs)

    # Stop monitoring
    done = True
    t.join()

    peak_mem_usage_gb = max(mem_usage) - baseline_mem
    return peak_mem_usage_gb


- **首先，我们追踪上一节** 使用 **顺序加载权重（Sequential Weight Loading）** 方法时的 **CPU 内存占用情况**。  

In [13]:
def load_sequentially():
    start_memory_tracking()

    model = GPTModel(BASE_CONFIG).to(device)

    state_dict = torch.load("model.pth", map_location="cpu", weights_only=True)

    print_memory_usage()

    # Sequentially copy weights to the model's parameters
    with torch.no_grad():
        for name, param in model.named_parameters():
            if name in state_dict:
                param.copy_(state_dict[name].to(device))
            else:
                print(f"Warning: {name} not found in state_dict.")

    print_memory_usage()


peak_memory_used = memory_usage_in_gb(load_sequentially)
print(f"-> Maximum CPU memory allocated: {peak_memory_used:.1f} GB")

Maximum GPU memory allocated: 6.4 GB
Maximum GPU memory allocated: 6.7 GB
-> Maximum CPU memory allocated: 6.3 GB


- **假设我们使用的设备 CPU 内存较小，但 GPU 内存充足**。  
- 我们可以 **利用 PyTorch 的 `"meta"` 设备**，在 **CPU 和 GPU 内存占用之间进行权衡**。  
- **PyTorch 的 `"meta"` 设备** 是一种特殊的设备类型，它允许创建 **不实际分配内存** 的张量，即 **"meta" 张量**。  
- 这对于 **模型分析（Model Analysis）或架构定义（Architecture Definition）** 等任务非常有用，在这些任务中，我们只需要 **张量的形状和数据类型**，而不需要 **真正的内存分配**。  

In [14]:
def load_sequentially_with_meta():
    start_memory_tracking()

    with torch.device("meta"):
        model = GPTModel(BASE_CONFIG)

    model = model.to_empty(device=device)

    state_dict = torch.load("model.pth", map_location=device, weights_only=True)

    print_memory_usage()

    # Sequentially copy weights to the model's parameters
    with torch.no_grad():
        for name, param in model.named_parameters():
            if name in state_dict:
                param.copy_(state_dict[name])
            else:
                print(f"Warning: {name} not found in state_dict.")

    print_memory_usage()

peak_memory_used = memory_usage_in_gb(load_sequentially_with_meta)
print(f"-> Maximum CPU memory allocated: {peak_memory_used:.1f} GB")

Maximum GPU memory allocated: 12.8 GB
Maximum GPU memory allocated: 12.8 GB
-> Maximum CPU memory allocated: 1.3 GB


- 如上所示，通过 **在 `"meta"` 设备上创建模型，并直接将权重加载到 GPU 内存**，我们 **显著降低了 CPU 内存占用**。  
- 这可能引发一个问题：**“那么，顺序加载权重（Sequential Weight Loading）是否仍然有必要？它与原始方法相比效果如何？”**  
- 接下来，我们对比 **PyTorch 的标准权重加载方法**（即本笔记本开头的 **初始权重加载方式**），看看它的 CPU 和 GPU 内存占用情况。  

In [15]:
def baseline():
    start_memory_tracking()

    model = GPTModel(BASE_CONFIG)
    model.to(device)

    model.load_state_dict(torch.load("model.pth", map_location=device, weights_only=True))
    model.to(device)
    model.eval();

    print_memory_usage()

peak_memory_used = memory_usage_in_gb(baseline)
print(f"-> Maximum CPU memory allocated: {peak_memory_used:.1f} GB")

Maximum GPU memory allocated: 12.8 GB
-> Maximum CPU memory allocated: 4.4 GB


- 如上所示，**不使用 `"meta"` 设备的“简单”权重加载方式，占用了更多的内存**。  
- 换句话说，**如果设备的 CPU 内存有限**，可以采用 **`"meta"` 设备方法**，直接将模型权重加载到 **GPU 内存**，从而 **降低 CPU 内存峰值占用**。  

&nbsp;
## **6. 使用 `mmap=True`（推荐方法）**  

- **如果你是 PyTorch `torch.load` 的中高级用户**，可能会想知道这些方法与 **`mmap=True` 选项** 有何区别。  
- **`mmap=True`** 选项 **启用了内存映射文件 I/O（Memory-Mapped File I/O）**，使得张量可以 **直接从磁盘访问数据**，而不需要将整个文件加载到 RAM 中，从而在 **RAM 受限时显著降低内存占用**。  
- 另请参考 **[mikaylagawarecki 的有用评论](https://github.com/rasbt/LLMs-from-scratch/issues/402)**。  
- 乍一看，`mmap=True` **可能比前面介绍的顺序加载方法效率更低**：  

In [37]:
def best_practices():
  with torch.device("meta"):
      model = GPTModel(BASE_CONFIG)

  model.load_state_dict(
      torch.load("model.pth", map_location=device, weights_only=True, mmap=True),
      assign=True
  )

  print_memory_usage()

peak_memory_used = memory_usage_in_gb(best_practices)
print(f"-> Maximum CPU memory allocated: {peak_memory_used:.1f} GB")

Maximum GPU memory allocated: 6.4 GB
-> Maximum CPU memory allocated: 5.9 GB


- **当前 CPU RAM 占用较高的原因**，是因为 **该设备的 CPU 内存充足**，因此 PyTorch 默认将模型加载到 RAM 中。  
- 但是，**如果设备的 CPU RAM 受限**，`mmap` 方法 **会显著降低内存使用量**，因为它允许张量 **直接从磁盘访问数据，而无需将整个文件加载到 RAM**。  

&nbsp;
## 7. 其他方法

- 本笔记本主要介绍 **PyTorch 内置的简单方法**，用于高效加载模型权重。  
- **在 CPU 内存受限的情况下**，推荐使用 **`mmap=True` 方法**（前文已详细介绍）。  
- 另外，还有一种 **“暴力”方法**：即 **将每个权重张量（Tensor）分别存储和加载**，以减少内存占用。  

In [13]:
model = GPTModel(BASE_CONFIG)
# Assume `model` is your trained model
state_dict = model.state_dict()

# Create a directory to store individual parameter files
os.makedirs("model_parameters", exist_ok=True)

# Save each parameter tensor separately
for name, param in state_dict.items():
    torch.save(param.cpu(), f"model_parameters/{name}.pt")

del model

In [16]:
def load_individual_weights():

    start_memory_tracking()

    with torch.device("meta"):
        model = GPTModel(BASE_CONFIG)

    model = model.to_empty(device=device)

    print_memory_usage()
    param_dir = "model_parameters"

    with torch.no_grad():
        for name, param in model.named_parameters():
            weight_path = os.path.join(param_dir, f"{name}.pt")
            if os.path.exists(weight_path):
                param_data = torch.load(weight_path, map_location="cpu", weights_only=True)
                param.copy_(param_data)
                del param_data  # Free memory
            else:
                print(f"Warning: {name} not found in {param_dir}.")

    print_memory_usage()


peak_memory_used = memory_usage_in_gb(load_individual_weights)
print(f"-> Maximum CPU memory allocated: {peak_memory_used:.1f} GB")

Maximum GPU memory allocated: 6.4 GB
Maximum GPU memory allocated: 6.4 GB
-> Maximum CPU memory allocated: 0.3 GB
